In [143]:
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Tuple

from models import ModelParams

# 1. IO table

In real data an IO table is represented in 2-D with three compenents:
- matrix of the  shape $NJ \times NJ$ for the intermedate goods consumption, 
- matrix of the shape $NJ \times N$ for the final goods consumption, and 
- matrix of the shape $NJ$ for the value-added. 

Given $N$ and $J$, we do the follows:

1. read the real data from the excel sheet to get the three matrix named $M$, $F$ and $V$
2. transformation:
- transform the $M$ matrix to the shape $(N, J, N, J)$ with the index meaning (export country, export sector, import country, import sector) 
- transform the $F$ matrix to the shape $(N, J, N)$ with the index meaning (export country, export sector, import country)
- transform the $V$ matrix to the shape $(N, J)$ with the index meaning (country, sector)

In [144]:
def read_io_excel(
    file_path: str | Path,
    N: int,
    J: int,
    *,
    sheets: Tuple[str, str, str] = ("M", "F", "V"),
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Read an IO table stored in three sheets (M, F, V) of an Excel file.

    Parameters
    ----------
    file_path : str or Path
        Location of the workbook.
    N, J : int
        Number of countries and sectors.
    sheets : tuple of str, default ("M", "F", "V")
        The sheet names that contain the intermediate-goods matrix,
        the final-goods matrix, and the value-added vector.

    Returns
    -------
    M : ndarray, shape (N·J, N·J)
    F : ndarray, shape (N·J, N)
    V : ndarray, shape (N·J,)
    """
    file_path = Path(file_path)

    # ── 1. read the three sheets ────────────────────────────────────────────────
    M = pd.read_excel(file_path, sheet_name="M", header=None).to_numpy(float)
    F = pd.read_excel(file_path, sheet_name="F", header=None).to_numpy(float)
    V = pd.read_excel(file_path, sheet_name="V", header=None).to_numpy(float).ravel()


    # ── 2. sanity-check the raw shapes ─────────────────────────────────────────
    if M.shape != (N * J, N * J):
        raise ValueError(f"M sheet shape {M.shape} ≠ {(N*J, N*J)}")
    if F.shape != (N * J, N):
        raise ValueError(f"F sheet shape {F.shape} ≠ {(N*J, N)}")
    if V.size != N * J:
        raise ValueError(f"V sheet length {V.size} ≠ {N*J}")

    return M, F, V

In [145]:
data_path = "data/toy_IO.xlsx"
N = 2
J = 2
M_2d, F_2d, V_2d = read_io_excel(data_path, N, J)

In [146]:
# (export country, export sector, import country, import sector)
M= M_2d.reshape(N, J, N, J) 
# (export country, export sector, import country)
F = F_2d.reshape(N, J, N)
# (country, sector)
V = V_2d.reshape(N, J)
# spot-check: element (e=2,s=1,i=1,t=2) should match the 2-D origin
e, s, i, t = 2, 1, 1, 2     # 1-based labels
assert (
    M[e-1, s-1, i-1, t-1] ==
    M_2d[(e-1)*J + (s-1), (i-1)*J + (t-1)]
)

# 2. Derived Variables

- **The trade-deficit matrix** $TD$ with the shape $(N, )$ is the calculated as 

$$TD = IM - EX$$

- **The income matrix** $Y$ with the shape $(N, )$ is calculated as 

$$Y = \sum_j^J V + TD$$
- **The expenditure-share matrix** $\boldsymbol{\alpha}$ with the shape $(N,J)$ is defined element-wise as

$$
\alpha_{n j}
\;=\;
\frac{\displaystyle\sum_{e=1}^{N} F_{e j n}}
     {\displaystyle\sum_{j'=1}^{J}\sum_{e=1}^{N} F_{e j' n}},
\qquad
n = 1,\dots,N,\;
j = 1,\dots,J.
$$

- **The value-added share matrix** $\boldsymbol{\beta}$, shape $(N,J)$  

  $$
  \beta_{n j}
  \;=\;
  \frac{V_{n j}}
       {\text{EX}_{n j}},
  \qquad
  n = 1,\dots,N,\;
  j = 1,\dots,J,
  $$

  where the sector-level gross output  

  $$
  \text{EX}_{n j}
  \;=\;
  \sum_{i=1}^{N}\sum_{t=1}^{J} M_{n j i t}  \;+\;
  \sum_{i=1}^{N} F_{n j i},
  $$

- **Intermediate-input share matrix** $\boldsymbol{\gamma}$, shape $(N,J,J)$  

  $$
  \gamma_{n j k}
  \;=\;
  \frac{\displaystyle\sum_{e=1}^{N}  M_{e\,k\,n\,j}}
       {\displaystyle\sum_{e=1}^{N}\sum_{t=1}^{J} M_{e\,t\,n\,j}},
  \qquad
  n = 1,\dots,N,\;
  j,k = 1,\dots,J.
  $$

- **Final-goods import-share tensor** $\boldsymbol{\pi}^{F}$ (`pif`), shape $(N,N,J)$  

  $$
  \pi^{F}_{n i k}
  \;=\;
  \frac{F_{\,i k n}}
       {\displaystyle\sum_{e=1}^{N} F_{\,e k n}},
  \qquad
  n,i = 1,\dots,N,\;
  k   = 1,\dots,J.
  $$  


- **Intermediate-goods import-share tensor** $\boldsymbol{\pi}^{M}$ (`pim`), shape $(N,N,J)$  

  $$
  \pi^{M}_{n i k}
  \;=\;
  \frac{\displaystyle\sum_{j=1}^{J} M_{\,i k n j}}
       {\displaystyle\sum_{e=1}^{N}\sum_{j=1}^{J} M_{\,e k n j}},
  \qquad
  n,i = 1,\dots,N,\;
  k   = 1,\dots,J.
  $$  

- **Combined-import share tensor** $\boldsymbol{\pi}^{\text{all}}$ (`piall`), shape $(N,N,J)$  

  $$
  \pi^{\text{all}}_{n i k}
  \;=\;
  \dfrac{\displaystyle
         F_{\,i k n}
         \;+\;
         \sum_{j=1}^{J} M_{\,i k n j}}
        {\displaystyle
         \sum_{e=1}^{N}\Bigl[
             F_{\,e k n}
             \;+\;
             \sum_{j=1}^{J} M_{\,e k n j}
         \Bigr]},
  \qquad
  n,i = 1,\dots,N,\;
  k   = 1,\dots,J.
  $$


 - **Final-goods expenditure matrix** $\mathbf{X}^{F}$, shape $(N,J)$  

  $$
  X^{F}_{n j}
    \;=\;
    \alpha_{n j}\;Y_n,
    \qquad
    n = 1,\dots,N,\;
    j = 1,\dots,J,
  $$


- **Intermediate-goods expenditure matrix** $\mathbf{X}^{M}$, shape $(N,J)$  

  $$
  X^{M}_{n j}
    \;=\;
    \sum_{i=1}^{N}\sum_{t=1}^{J} M_{\,i j n t},
    \qquad
    n = 1,\dots,N,\;
    j = 1,\dots,J,
  $$


#

In [147]:
def calc_TD(
    M: np.ndarray,          # shape (N, J, N, J)
    F: np.ndarray           # shape (N, J, N)
) -> np.ndarray:
    """
    Compute imports, exports, and trade-deficit vector for each country.

    Parameters
    ----------
    M : ndarray (N, J, N, J)
        Intermediate-goods flows:
        (export country, export sector, import country, import sector)
    F : ndarray (N, J, N)
        Final-goods flows:
        (export country, export sector, import country)

    Returns
    -------
    imports  : ndarray (N,)   total goods absorbed by each country
    exports  : ndarray (N,)   total goods supplied by each country
    deficit  : ndarray (N,)   imports - exports  (positive ⇒ deficit)
    """
    # imports: keep importer axis (2), sum everything else
    imports = M.sum(axis=(0, 1, 3)) + F.sum(axis=(0, 1))

    # exports: keep exporter axis (0), sum everything else
    exports = M.sum(axis=(1, 2, 3)) + F.sum(axis=(1, 2))

    # deficit vector (TD)
    deficit = imports - exports
    return deficit

In [148]:
def calc_Y(
        V: np.ndarray,          # shape (N, J)
        TD: np.ndarray,          # shape (N,)
        ) -> np.ndarray:   
    """
    Compute the total output vector for each country.

    Parameters
    ----------
    V : ndarray (N, J)
        Value-added vector:
        (country, sector)
    TD : ndarray (N,)
        Trade-deficit vector:
        (country)

    Returns
    -------
    Y : ndarray (N,)   total output of each country
    """
    # total output = value added + trade deficit
    Y = V.sum(axis=1) + TD
    return Y

In [149]:
def calc_alpha(F: np.ndarray) -> np.ndarray:
    """
    Compute α[n,j] = share of country-n final-goods spending that falls on good j.

    Parameters
    ----------
    F : ndarray, shape (N, J, N)
        Final-goods tensor  (export-country, export-sector, import-country).

    Returns
    -------
    alpha : ndarray, shape (N, J)
        Rows sum to one (up to floating error); NaNs if a country has zero final demand.
    """
    # ── 1. total spending on each good j in importer n ─────────────────────────
    #     sum over export-country axis 0  ⇒  shape (J, N), then transpose → (N,J)
    exp_nj = F.sum(axis=0).T             # (N,J)

    # ── 2. country totals ──────────────────────────────────────────────────────
    totals = exp_nj.sum(axis=1, keepdims=True)  # (N,1)

    # avoid divide-by-zero: if a country has no final demand, leave shares as 0
    with np.errstate(invalid="ignore", divide="ignore"):
        alpha = np.where(totals != 0, exp_nj / totals, 0.0)

    return alpha

In [150]:
def calc_beta(M, F, V):
    """
    Parameters
    ----------
    M : ndarray, shape (N, J, N, J)
        Intermediate-goods tensor.
    F : ndarray, shape (N, J, N)
        Final-goods tensor.
    V : ndarray, shape (N, J)
        Value-added by country & sector.

    Returns
    -------
    beta : ndarray, shape (N, J)
        Share of value-added in gross output.
    """
    # gross output from intermediate deliveries
    interm_out = M.sum(axis=(2, 3))        # (N, J)

    # gross output from final-goods deliveries
    final_out  = F.sum(axis=2)             # (N, J)

    X = interm_out + final_out             # gross output (N, J)

    # avoid divide-by-zero
    with np.errstate(invalid="ignore", divide="ignore"):
        beta = np.where(X != 0, V / X, 0.0)

    return beta

In [151]:
def calc_gamma(M: np.ndarray) -> np.ndarray:
    """
    Compute γ[n,j,k] = share of intermediate inputs from sector k
    in the production of sector j in country n.

    Parameters
    ----------
    M : ndarray, shape (N, J, N, J)
        Intermediate-goods tensor:
        (export country, export sector, import country, import sector).

    Returns
    -------
    gamma : ndarray, shape (N, J, J)
        Rows over k sum to one for each (n,j) pair.
    """
    # 1. Aggregate over exporter countries  →  flows[k, n, j]
    flows = M.sum(axis=0)                # shape (J, N, J)

    # 2. Re-order axes to (n, j, k)
    flows_njk = flows.transpose(1, 2, 0) # shape (N, J, J)

    # 3. Normalise by total inputs into each (n, j)
    denom = flows_njk.sum(axis=2, keepdims=True)  # (N, J, 1)

    with np.errstate(divide="ignore", invalid="ignore"):
        gamma = np.where(denom != 0, flows_njk / denom, 0.0)

    return gamma

In [152]:
def calc_pif(F: np.ndarray) -> np.ndarray:
    """
    Parameters
    ----------
    F : ndarray, shape (N, J, N)
        Final-goods tensor (exporter, sector, importer).

    Returns
    -------
    pif : ndarray, shape (N, N, J)
        π^F[importer n, exporter i, export-sector k]
    """
    # re-order to (importer n, exporter i, sector j)
    flows = F.transpose(2, 0, 1)             # (N, N, J)

    denom = flows.sum(axis=1, keepdims=True) # (N, 1, J)

    with np.errstate(divide="ignore", invalid="ignore"):
        pif = np.where(denom != 0, flows / denom, 0.0)

    return pif

In [153]:
def calc_pim(M: np.ndarray) -> np.ndarray:
    """
    Parameters
    ----------
    M : ndarray, shape (N, J, N, J)
        Intermediate-goods tensor
        (export country, export sector, import country, import sector).

    Returns
    -------
    pim : ndarray, shape (N, N, J)
          π^M[importer n, exporter i, export-sector k]
    """
    # sum over import-sector axis (3)  → (exporter i, export-sector k, importer n)
    flows_raw = M.sum(axis=3)               # (N, J, N)

    # reorder to (importer n, exporter i, export-sector k)
    flows = flows_raw.transpose(2, 0, 1)    # (N, N, J)

    denom = flows.sum(axis=1, keepdims=True)  # (N, 1, J)

    with np.errstate(divide="ignore", invalid="ignore"):
        pim = np.where(denom != 0, flows / denom, 0.0)

    return pim

In [154]:
def calc_piall(M: np.ndarray, F: np.ndarray) -> np.ndarray:
    """
    Combined final- and intermediate-goods import shares.

    Parameters
    ----------
    M : ndarray, shape (N, J, N, J)
        Intermediate-goods tensor (exporter, export-sector, importer, import-sector)
    F : ndarray, shape (N, J, N)
        Final-goods tensor         (exporter, export-sector, importer)

    Returns
    -------
    piall : ndarray, shape (N, N, J)
        π_all[importer n, exporter i, export-sector k]
    """
    # --- total flows of export-sector k from exporter i to importer n ----------
    interm = M.sum(axis=3)          # sum over import-sector j  →  (N, J, N)
    total  = interm + F             # (N, J, N)

    # reorder to (importer n, exporter i, export-sector k)
    flows = total.transpose(2, 0, 1)          # (N, N, J)

    denom = flows.sum(axis=1, keepdims=True)  # (N, 1, J)

    with np.errstate(divide="ignore", invalid="ignore"):
        piall = np.where(denom != 0, flows / denom, 0.0)

    return piall

In [155]:
def calc_Xf_Xm(alpha: np.ndarray,
               Y: np.ndarray,
               M: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Final- and intermediate-goods expenditure matrices.

    Parameters
    ----------
    alpha : ndarray, shape (N, J)
        Expenditure-share matrix.
    Y     : ndarray, shape (N,)
        Income vector by country.
    M     : ndarray, shape (N, J, N, J)
        Intermediate-goods tensor
        (export-country, export-sector, import-country, import-sector).

    Returns
    -------
    Xf : ndarray, shape (N, J)
         Final-goods expenditure  (Xf = alpha * Y).
    Xm : ndarray, shape (N, J)
         Intermediate-goods expenditure.
    """
    # Final-goods expenditure: broadcast Y across sectors
    Xf = alpha * Y[:, None]          # (N, J)

    # Intermediate-goods expenditure: sum over exporters (0) and import-sector (3)
    Xm = M.sum(axis=(0, 3)).T        # (N, J)

    return Xf, Xm

In [156]:
TD = calc_TD(M, F)
Y = calc_Y(V, TD)
alpha = calc_alpha(F)
beta = calc_beta(M, F, V)
gamma = calc_gamma(M)
pif = calc_pif(F)
pim = calc_pim(M)
piall = calc_piall(M, F)
Xf, Xm = calc_Xf_Xm(alpha, Y, M)



# 3. Additional Variables

We also need two variables to complete the construction of model parameters.

- $\theta$: shape $(N,)$ the trade elasticity of each country
- $\tilde{\tau}$: shape $(N, N, J)$, with indexing (import country, export country, export sector), the $1 + \text{tariff rate}$ on imports from country $i$ in sector $j$ of country $n$

In [157]:
theta_constant = 7
theta = np.ones(N) * theta_constant

tau_constant = 0
tilde_tau = np.ones((N, N, J)) + tau_constant

VA = np.sum(V, axis=1)  # (N,)
gamma *= 0.5
country_lists = {"A", "B"}
sector_lists = {"1", "2"}


In [158]:
params = ModelParams(
    N=N,
    J=J,
    alpha=alpha,
    beta=beta,
    gamma=gamma,
    theta=theta,
    pif=pif,
    pim=pim,
    tilde_tau=tilde_tau,
    Xf=Xf,
    Xm=Xm,
    VA=VA,
    D=TD,
    country_list=country_lists,
    sector_list=sector_lists,
)

In [161]:
piall

array([[[0.57142857, 0.58333333],
        [0.42857143, 0.41666667]],

       [[0.4       , 0.41666667],
        [0.6       , 0.58333333]]])